In [3]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.optim import SGD, Adam
from tqdm import tqdm
from utils import Encoder, Decoder, additive_white_gaussian_noise_channel

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [ ]:
CONFIG_TRAIN = {
    "antenna_config": "siso",  # Antenna configuration
    "channel_model": "rayleigh",  # Channel type
    "M": 16,  # Number of constellation points
    "flag_train_model": True,  # Flag to control training
    "training_snr": 12,  # Training SNR (dB)
}
CONFIG_TRAIN["best_encoder_path"] = (
    f"./model/ae_{CONFIG_TRAIN['antenna_config']}_{CONFIG_TRAIN['channel_model']}_{CONFIG_TRAIN['M']}qam_best_encoder.pth"
)
CONFIG_TRAIN["best_decoder_path"] = (
    f"./model/ae_{CONFIG_TRAIN['antenna_config']}_{CONFIG_TRAIN['channel_model']}_{CONFIG_TRAIN['M']}qam_best_decoder.pth"
)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
encoder = Encoder([CONFIG_TRAIN["M"], 64, 64, 64, 8]).to(device)
decoder = Decoder([8, 512, 512, 512, CONFIG_TRAIN["M"]]).to(device)

In [ ]:
checkpoint = torch.load(CONFIG_TRAIN["best_encoder_path"], map_location=device)
encoder.load_state_dict(checkpoint)

In [4]:
input = torch.randn(3, 5, requires_grad=True)

In [12]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
# input is of size N x C = 3 x 5
input = torch.randn(3, 5, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
output = loss(m(input), target)
output.backward()

In [13]:
m(input)

tensor([[-3.9215, -1.1202, -0.8376, -1.9857, -2.4781],
        [-1.4749, -1.3096, -2.6351, -2.2789, -1.1173],
        [-2.9245, -0.7828, -2.3251, -1.3161, -2.0937]],
       grad_fn=<LogSoftmaxBackward0>)

In [14]:
target

tensor([1, 0, 4])